# Practice №11
## by Anton Kondrashov
##### HSE, CS, BSE 141(1)

### Dependencies

In [24]:
%precision 6
%load_ext line_profiler
%load_ext autoreload
%autoreload 1
import sklearn
import pandas as pd
import numpy as np
import codecs
import json
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing

pd.options.display.max_colwidth=100
np.set_printoptions(linewidth=140,edgeitems=10)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Datasets

In [25]:
def unpack_dataset(path, to=None):
    dicts = []
    
    if to is not None:
        dicts = list(to.to_dict().values())
#     with codecs.open(path, 'r', encoding=encoding) as f:
#         for line in f:
#             dicts.append(eval(line))
    with open(path) as f:
        for line in f:
            single_dict = eval(line)
            dicts.append(single_dict)
    
    return pd.DataFrame(dicts)

def labelize(column_name, dataset):
    le = preprocessing.LabelEncoder()
    le.fit(dataset[column_name])
    dataset[column_name] = le.transform(dataset[column_name])
    
def process_runtime(time):
    if re.findall("\d*", time)[0] == '':
        return 0 
    else:
        return int(re.findall("\d*", time)[0])

# def replace_dates(dataset):
#     extension_set = pd.DataFrame(columns=['year', 'month', 'day'])
#     date['time'].apply(lambda x: x[:3])

In [26]:
main_path = '/Users/kondranton/Desktop/Datasets'
movie_descriptions = unpack_dataset(main_path + '/movie_descriptions.txt')

feedbacks = []
with open(main_path + '/train_feedback.txt') as f:
        for line in f:
            feedback = eval(line)
            feedbacks.append(feedback)

In [ ]:
with open(main_path + '/test_feedback.txt') as f:
        for line in f:
            feedback = eval(line)
            feedbacks.append(feedback)

In [27]:
feedbacks = pd.DataFrame(feedbacks)

### Preprocessing

In [4]:
# desc
def extract_year(date):
    nums = [int(x) for x in re.findall("\d*", date) if x!='']
    if len(nums) == 2:
        return nums[1]
    else:
        return 2000

movie_descriptions['runtime'] = movie_descriptions['runtime'].apply(process_runtime)
labelize('rated', movie_descriptions)
labelize('type', movie_descriptions)
labelize('genre', movie_descriptions)
labelize('language', movie_descriptions)
labelize('time', feedbacks)

In [28]:
Z=pd.merge(feedbacks,movie_descriptions,on='MovieId')

In [6]:
np.random.seed(0)
Z = Z.sample(frac=1.0).reset_index(drop=True)

In [36]:
np.random.seed(0)

index = np.arange(len(Z))

mask = index < 0.6 * len(Z)
Ztr = Z[mask]
Zte = Z[~mask]

In [35]:
Z.rename(columns={'score':'y'}, inplace=True)

In [8]:
train_sels = ~np.isnan(Z.y)
test_sels = np.isnan(Z.y)

train_inds = np.where(train_sels)
test_inds = np.where(test_sels)

del train_sels, test_sels

In [9]:
corpus = Z.text +  Z.synopsis + Z.writer + Z.summary + Z.director

In [41]:
vectorizer = TfidfVectorizer(min_df=1, stop_words='english')
X = vectorizer.fit_transform(corpus)
Y=Ztr.y.values

In [42]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer(sparse=True)
X1 = v.fit_transform(Ztr[['UserId', 'rated', 'runtime', 'year']].to_dict(orient='records'))

In [43]:
from scipy.sparse import hstack
X = hstack([X1, X])#.toarray()

In [44]:
param_grid = {'C': [0.1, 0.03, 1, 3, 5, 7]}
clf = sklearn.model_selection.GridSearchCV(sklearn.svm.LinearSVC(),param_grid, n_jobs=-1)
# clf.fit(X[train_inds], Y[train_inds])
clf.fit(X,Y)

GridSearchCV(cv=None, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': [0.1, 0.03, 1, 3, 5, 7]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [45]:
clf.best_params_

{'C': 5}

In [51]:
Zte[[x for x in Zte.columns if x != 'y']]

,MovieId,UserId,id,profileName,summary,text,time,actors,awards,country,director,genre,language,plot,rated,released,runtime,synopsis,type,writer,year
40172,2586,1826,14848,Nys04,Best Movie of the Year,Best movie I have seen this year .....nuff said.....oh erm just behind The Dark Knight of course...,2008-11-01 03:00:00,"Robin Williams, Maria Conchita Alonso, Cleavant Derricks, Alejandro Rey",Nominated for 1 Golden Globe. Another 1 nomination.,USA,Paul Mazursky,"Comedy, Drama, Romance","English, Russian","A Russian circus visits the US. A clown wants to defect, but doesn't have the nerve. His saxopho...",R,06 Apr 1984,115 min,\n\nSet against the backdrop of the Cold War and Soviet political repression of the early 1980s ...,movie,"Paul Mazursky, Leon Capetanos",1984
40173,2586,14946,86567,"Volasar ""audiophile""",offensive and a waste of time,the war sequence was completely offensive...there is nothing funny about people getting shot and...,2009-02-21 03:00:00,"Robin Williams, Maria Conchita Alonso, Cleavant Derricks, Alejandro Rey",Nominated for 1 Golden Globe. Another 1 nomination.,USA,Paul Mazursky,"Comedy, Drama, Romance","English, Russian","A Russian circus visits the US. A clown wants to defect, but doesn't have the nerve. His saxopho...",R,06 Apr 1984,115 min,\n\nSet against the backdrop of the Cold War and Soviet political repression of the early 1980s ...,movie,"Paul Mazursky, Leon Capetanos",1984
40174,2586,3635,8696,"Mike M ""Mike M""",Great Over-The-Top Rude Romp is not for all!,"Rude, hilarious, politically incorrect, action packed, gut busting laugh fest! I laughed so hard...",2009-06-21 03:00:00,"Robin Williams, Maria Conchita Alonso, Cleavant Derricks, Alejandro Rey",Nominated for 1 Golden Globe. Another 1 nomination.,USA,Paul Mazursky,"Comedy, Drama, Romance","English, Russian","A Russian circus visits the US. A clown wants to defect, but doesn't have the nerve. His saxopho...",R,06 Apr 1984,115 min,\n\nSet against the backdrop of the Cold War and Soviet political repression of the early 1980s ...,movie,"Paul Mazursky, Leon Capetanos",1984
40175,2586,4449,37188,Wesley Mullins,Entourage-like hollywood satire is funniest film of the year,A group of prima donna actors and incompetent filmmakers travel to Vietnam to adapt a gritty sto...,2009-01-04 03:00:00,"Robin Williams, Maria Conchita Alonso, Cleavant Derricks, Alejandro Rey",Nominated for 1 Golden Globe. Another 1 nomination.,USA,Paul Mazursky,"Comedy, Drama, Romance","English, Russian","A Russian circus visits the US. A clown wants to defect, but doesn't have the nerve. His saxopho...",R,06 Apr 1984,115 min,\n\nSet against the backdrop of the Cold War and Soviet political repression of the early 1980s ...,movie,"Paul Mazursky, Leon Capetanos",1984
40176,2586,7558,35243,P. Oski,Vulgar and Offensive...,"... and mostly a riot! Tropic Thunder fails, only insofar as it falls short of all its ambitions...",2008-11-28 03:00:00,"Robin Williams, Maria Conchita Alonso, Cleavant Derricks, Alejandro Rey",Nominated for 1 Golden Globe. Another 1 nomination.,USA,Paul Mazursky,"Comedy, Drama, Romance","English, Russian","A Russian circus visits the US. A clown wants to defect, but doesn't have the nerve. His saxopho...",R,06 Apr 1984,115 min,\n\nSet against the backdrop of the Cold War and Soviet political repression of the early 1980s ...,movie,"Paul Mazursky, Leon Capetanos",1984
40177,2586,13756,32083,C. HAWKINS,Very Funny Tom Cruise and Robert Downey were great.,"The whole cast was really good. I have watched this DVD probably five times already, and liked ...",2009-10-13 03:00:00,"Robin Williams, Maria Conchita Alonso, Cleavant Derricks, Alejandro Rey",Nominated for 1 Golden Globe. Another 1 nomination.,USA,Paul Mazursky,"Comedy, Drama, Romance","English, Russian","A Russian circus visits the US. A clown wants to defect, but doesn't have the nerve. His saxopho...",R,06 Apr 1984,115 min,\n\nSet against the backdrop of the Cold War and Soviet political repression of the ea

In [49]:
Y_hat = clf.predict(Zte[[x for x in Zte.columns if x != 'y']])

ValueError: could not convert string to float: '1978–1990'

In [50]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [23]:
#output
Y_hat = clf.predict(X[test_inds])
content = pd.concat([pd.DataFrame(Z.id.values[test_inds]), pd.DataFrame(Y_hat)], axis=1)
content.columns = ['ReviewId', 'Score']
solution = pd.DataFrame(content, dtype=int)
solution.to_csv(main_path + '/solution.csv',index=False)

In [52]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

ImportError: No module named 'xgboost'

print('sdf')

In [ ]:
param_grid = {'C': [0.1, 0.03, 1, 3, 5, 7]}
clf = sklearn.model_selection.GridSearchCV(sklearn.svm.LinearSVC(),param_grid, n_jobs=-1)
# clf.fit(X[train_inds], Y[train_inds])
clf.fit(X,Y)

In [ ]:
import platform
print(platform.python_version())

### Conclusion

As we can see, these words have the most significant impact on a review score. The words express someone's opinion, so they can be intepreted as a movie empirical score. 